<a href="https://colab.research.google.com/github/Kpantigere/Batch_Fall_23_DataA_nalytics/blob/main/Copy_of_Bigquery_supermarket_fall_2023_Ibrahim_u_m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL in Bigquery and Google Colab.

In order to run a query in Bigquery we can always do it using the front end.

Nevertheless, sometimes we prefer to run the query from other platforms, this way we can store the queries in a "jupiter notebook" like system.

This is what we will be doing here. We will connect to our database in Bigquery, and then run queries from our google colab.


The next cell will ask for your google credentials, please use the same ones where you have your bigquery access.

In [2]:
from google.colab import auth
auth.authenticate_user()

Now that we are logged in, we can start running queries. We will ask to be connected to a specific project:


In [3]:
from google.cloud import bigquery
client = bigquery.Client(project='redi-da-fall-2023')

Once we are connected to the project, we can run queries in the following way:

In [4]:
sql_query = ('''

SELECT

*

FROM supermarket_redi.sales

LIMIT 1000


''')

df = client.query(sql_query).to_dataframe()

In [ ]:
df.head()

Explore the next sql functions:


How to explore just one column:

In [6]:
sql_query = ('''

SELECT
item_code
FROM supermarket_redi.sales
LIMIT 1000

''')

df = client.query(sql_query).to_dataframe()

In [ ]:
df.head()

How to explore just the DISTINCT (or unique) values of one column:

In [8]:
sql_query = ('''

SELECT
DISTINCT
item_code
FROM supermarket_redi.sales
LIMIT 1000

''')

df = client.query(sql_query).to_dataframe()

In [ ]:
df.head()

select the whole dataset using where


In [10]:
sql_query = ('''

SELECT
*
FROM supermarket_redi.sales
WHERE Discount__Yes_No_ = true

''')

df = client.query(sql_query).to_dataframe()

In [ ]:
df.head()


select from a list of values using where in


In [12]:
sql_query = ('''

Select
  *
FROM supermarket_redi.sales

WHERE Item_Code IN (102900011032848,102900011031582)

''')

df = client.query(sql_query).to_dataframe()

In [ ]:
df.head()

USING GROUP BY

group by is a way to aggregate data

for example, now we want to know how many times an item has been sold (not how many sales but how many times that item appears on our table)


In [ ]:
sql_query = ('''

Select
  item_code,
  count(1) as nb_appearances
FROM supermarket_redi.sales
GROUP BY item_code

''')

df = client.query(sql_query).to_dataframe()

We can also subsitute GROUP BY item_code by GROUP BY 1 (meaning, to group by the first column of our new table)

This result by itself is not very useful, no?

Try to now add at the end of the query ORDER BY nb_appearances DESC



In [ ]:
sql_query = ('''

Select
  item_code,
  count(1) as nb_appearances
FROM supermarket_redi.sales

GROUP BY item_code
ORDER BY nb_appearances desc

''')

df = client.query(sql_query).to_dataframe()

now we have the top items that sold the most times, but lets see the item that sold actually the most kilos:


In [ ]:
sql_query = ('''


Select
  item_code,
  sum(Quantity_Sold__kilo_) as nb_kilos_sold
FROM supermarket_redi.sales

GROUP BY item_code
ORDER BY nb_kilos_sold desc

''')

df = client.query(sql_query).to_dataframe()

Calculations over a given dimension are done using the GROUP BY formula.

the calculations we can do are:

min(), max(), sum(), count() and count(distinct)

there are many others, but those are the most important ones.




In [ ]:
sql_query = ('''


SELECT
  date,
  item_code,
  min(Quantity_Sold__kilo_) as min_kilo_sold,
  max(Quantity_Sold__kilo_) as max_kilo_sold,
  sum(Quantity_Sold__kilo_ * Unit_Selling_Price__RMB_kg_) as revenue_generated

from supermarket_redi.sales
group by 1,2

''')

df = client.query(sql_query).to_dataframe()

DATE FUNCTIONS
-- https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions

date trunc: date trunc, will cut the date up to a certan point, if we chose week, it will flat the date to the starting week day, if we chose month, it will do the same but to the first day of the month.



In [ ]:
sql_query = ('''


SELECT
  date_trunc(date, week) as date_week,
  item_code,
  min(Quantity_Sold__kilo_) as min_kilo_sold,
  max(Quantity_Sold__kilo_) as max_kilo_sold,
  sum(Quantity_Sold__kilo_ * Unit_Selling_Price__RMB_kg_) as revenue_generated

from supermarket_redi.sales
group by 1,2

''')

df = client.query(sql_query).to_dataframe()

EXTRACT


Extract will take out one part of the date, for example the day of the month, or the month, or the day of the week.



In [ ]:
sql_query = ('''


SELECT
  extract(week from date) as calendar_week,
  item_code,
  min(Quantity_Sold__kilo_) as min_kilo_sold,
  max(Quantity_Sold__kilo_) as max_kilo_sold,
  sum(Quantity_Sold__kilo_ * Unit_Selling_Price__RMB_kg_) as revenue_generated

from supermarket_redi.sales
group by 1,2

''')

df = client.query(sql_query).to_dataframe()

CASE WHEN       

CASE WHEN is similar to the IF formula on other languages:

CASE WHEN  Discount__Yes_No_ IS TRUE THEN 'Item had discount' else 'Item did
not have discount' end as name_column,

CASE WHEN logic                 THEN ____                     ELSE ____                         END AS name_column,


EXERCISES:

What was the day that had the most revenue?

What was the day that had the least revenue?

Could you get the distribution of total kilos sold per day of the week? So, how many kilos were sold on a monday, how many on a tuesday...

Could you get the distribution of total kilos sold per hour of the day?So, how many kilos were sold at 9 am, how many at 10 am...

What was the item that generated the most sales?

Does the supermarket sell more during the morning (before 12 pm) or during the afternoon, evening?

What is the hour of the week that the supermarket sells the most?

What was the item that more times was discounted?
